In [1]:
#FACCIO LA STESSA COSA DEL FINE NON "upd", MA QUI INSERISCO TUTTE LE FUNZIONI "PARAMETRICHE", POICHÈ ANDRÒ A FAR AUMENTARE
#LA DIMENSIONE DELLE MATRICI E ANCHE DEL GF CON IL NUMERO DI ITERAZIONE, PER FARE DELLE STESSE VERIFICHE

In [2]:
!pip install galois
!pip install scipy


/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found
/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [3]:
pip install sympy


Note: you may need to restart the kernel to use updated packages.


In [57]:
import galois
import numpy as np
import time
import os
from scipy import stats
import csv
from sympy import isprime

In [58]:
#number of iteration for Montecarlo simulatiom
num_iterations = 200

In [59]:
#COMMON FUNCTIONS
def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

def make_bin_matrix(dim, field):
    bin_matrix = np.zeros(dim, dtype=int)
    perm = np.random.permutation(dim[0])
    for i in range(dim[0]):
        bin_matrix[i, perm[i]] = 1
    return field(bin_matrix)

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    #print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    """for raw in sorted_matrix:
        print(raw)
    print()"""
    return sorted_matrix, num_swap_columns


In [60]:
#FUNCTION FOR SORT DICTIONARY COMPLETE
# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row(matrix, error_counter, field_order, max_column):
    #operations = 0
    vector_of_counts = []  
    n_colonne = matrix.shape[1]  
    start = time.perf_counter()
    #MUST pass as "matrix" an np.ndarray 
    #matrix_int = matrix.view(np.ndarray)
    
    for row in matrix:
        counts = [0] * (field_order+1)  
        for value in row:
            #print(value)
            counts[value] += 1
            #operations += 1
        vector_of_counts.append(counts)

    vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        #print(vector_of_counts)
        print("Duplicated rows all columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(n_colonne, -1, -1)])[::-1]
    end = time.perf_counter()
    duration = (end - start) * 1000    
     
    #operations1 = 0
    vector_of_counts1 = []
    start = time.perf_counter()
    #matrix_int = matrix.view(np.ndarray)  
    #print(matrix_int)
    for row in matrix:
        #make a small array
        counts1 = [0] * max_column
        for value in row:
            #count only value < max_column
            if value < max_column:
                counts1[value] += 1
                #operations1 += 1
        #print(counts1)
        vector_of_counts1.append(counts1)
        #print(vector_of_counts1)
    #print(vector_of_counts1)
    vector_of_counts1 = np.array(vector_of_counts1)
    if has_duplicate_rows(vector_of_counts1):
        error_counter += 1
        #print(vector_of_counts1)
        print("Duplicated rows less columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    sorted_index1 = np.lexsort([vector_of_counts1[:, i] for i in range(vector_of_counts1.shape[1]-1, -1, -1)])[::-1]
    
    """for raw in matrix[sorted_index1]:
        print(np.sort(raw))"""
    #print()
    end = time.perf_counter()
    duration1 = (end - start) * 1000 
    print(np.array_equal(matrix[sorted_index], matrix[sorted_index1]))
    if np.array_equal(matrix[sorted_index], matrix[sorted_index1]):
        print("If positivo")
        return matrix[sorted_index], duration, 0, matrix[sorted_index1], duration1, 0, error_counter
    else:
        print("If negativo")
        error_counter += 1
        print(error_counter)
        return [], duration, 0, [], duration1, 0, error_counter
    
def algorithm_test1(matrix1, error_counter, k, field_order, max_column):
    row1_sort, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = order_row(matrix1, error_counter, field_order, max_column)
    #print(row1_sort)
    if len(row1_sort) == 0:
        return False, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    #print(matrix1_sorted)
    return True, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter


In [61]:
#FUNCTION FOR ORDINAMENTO REALE
#bette but cannot count operations
def ordina_matrice_ottimizzata(matrice, colonne_max):
    print("Entro in ordina matrice ottimizzata")
    
    n = matrice.shape[0]
    n_colonne = matrice.shape[1]
    
    # Array indici originali
    indici = np.arange(n)
    indici1 = np.arange(n)

    start = time.perf_counter()
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    matrice1 = ordinato

    indici = np.lexsort([matrice1[:, i] for i in range(n_colonne - 1, -1, -1)]) 
    
    #check duplicate rows 
    if has_duplicate_rows(matrice1[indici]):
        tempo = (end - start) * 1000
        print("Algoritmo ha almeno due righe uguali --> fallisco")
        return False, [], 0, 0, tempo, 0.0
    end = time.perf_counter()
    tempo = (end - start) * 1000
    
    #sorting with limited columns
    start = time.perf_counter()
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    matrice2 = ordinato
    #lexsort implemnets a timesort algorithm thet is better than a quick sort in worst case
    indici1 = np.lexsort([matrice2[:, i] for i in range(colonne_max - 1, -1, -1)])
    #check equality between two index
    if has_duplicate_rows(matrice2[indici1]):
        tempo1 = (end - start) * 1000
        print("Algoritmo ha almeno due righe uguali --> fallisco")
        return False, [], 0, 0, tempo, tempo1
    end = time.perf_counter()
    tempo1 = (end - start) * 1000
    if np.array_equal(matrice1[indici], matrice2[indici1]):
        #se le due matrici ottenute dai due ordinamenti sono uguali allora torno true, indici, numero di confronti e tempi
        #print(matrice1[indici], matrice2[indici1])
        return True, indici, 0, 0, tempo, tempo1
    else:
        print("Non sono uguali")
        return False, [], 0, 0, tempo, tempo1
    
#Funzione per ordinare le righe con contatori di swap. In num_confronti metto confronti in # sufficienti metto confronti1. 
def ordina_per_riga_con_contatori(matrice, num_swap_righe, max_column):
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    print("Funzione ordina_per_riga")
    posizioni_originali = np.arange(matrice.shape[0])
        
    ris_confronto, indici_ordinamento, num_confronti,conta_sufficienti, tempo, tempo1 = ordina_matrice_ottimizzata(matrice, max_column)

    #se torno true quindi due matrici sono uguali
    if ris_confronto:
        # Conta gli swap solo quando la posizione delle righe cambia
        for i, indice in enumerate(indici_ordinamento):
            if indice != posizioni_originali[i]:
                num_swap_righe += 1
        return matrice[indici_ordinamento], num_swap_righe, num_confronti, conta_sufficienti, tempo, tempo1
    else:
        return [], num_swap_righe, num_confronti, conta_sufficienti, tempo, tempo1

def test_algoritmo(matrice1, num_swap_righe, num_swap_colonne, k, max_column):
    print("Funzione test uguaglianza")
    start_time = time.perf_counter()
    #intervallo corretto per le righe
    righe = range(k)      
    
    print("Faccio ordina per riga con contatori")
    ordinata1, num_swap_righe, num_confronti, conta_sufficienti, tempo, tempo1 = ordina_per_riga_con_contatori(matrice1, 
                                                                num_swap_righe, max_column)
    print("Ordino le colonne")
    #se ordinata è praticamente vuoto vuol dire che ordina per riga per contatori non ha passato qualche test
    if len(ordinata1) == 0:
        end_time = time.perf_counter()
        durata = (end_time - start_time)*1000
        return False, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata, tempo, tempo1
    

    cf1, num_swap_colonne = sort_by_column(ordinata1, range(k), num_swap_colonne)
    end_time = time.perf_counter()
    durata = (end_time - start_time)*1000
    return True, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata, tempo, tempo1

In [62]:
def montecarlo_simulation(num_iterations):
    k = 25 #rows
    n_k = 25 #columns
    duration1_accumulator = 0
    duration1_2_accumulator = 0
    operations_accumulator = 0
    operations1_accumulator = 0
    simulazione_accettabile = 0
    accumula_durata_tutte_colonne = 0
    accumula_durata_meno_colonne = 0
    num_confronti_accumulatore = 0
    conta_sufficienti_accumulatore = 0
    error_counter = 0
    num_swap_righe = 0
    num_swap_colonne = 0

    csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_seconda_revisione/dati_differences_upd1.csv'
    #there can change variable name based on what u want
    file_exists = os.path.exists(csv_file_path)
    #there can change variable name based on what u want
    with open('dati_differences_upd1.csv', mode='a', newline='') as file:
        writer_csv = csv.writer(file)
        if not file_exists:
            writer_csv.writerow(['File','Iteration', 'Max_columns', 'Duration_w_all_Columns', 'Duration_w_max_columns', 'Num_op_w_all_columns', 'Num_op_w_max_columns', 'Round', 'Rows-Column'])

        for i in range(100):
            print(i)
            k += 1
            n_k += 1
            order = k+1
            while(not isprime(order)):
                order += 1
                k += 1
                n_k += 1
            field = galois.GF(order)
            print("Field order:", field.order)
            #cosider before 9% of the total columns and after 10% of the total columns (9% of 126 = 11, 10% of 126 = 12)
            max_column = int(k / 100 * 9)
            for j in range(num_iterations):
                A = field(np.random.randint(0, field.order, size=(k, n_k))) 
                while(has_duplicate_rows(A)):
                    print("Regenerate")
                    A = field(np.random.randint(0, field.order, size=(k, n_k))) 
                print(k, n_k, max_column)
                matrix_int = A.view(np.ndarray)
                
                check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = algorithm_test1(A, error_counter, k, field.order, max_column)
                esito, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata, tempo, tempo1 = test_algoritmo(A, num_swap_righe, num_swap_colonne, k, max_column)
                if check and esito: #write only if both return a sorted matrix
                    print("Check:", check, "Esito:", esito)
                    duration1_accumulator += duration1_1
                    duration1_2_accumulator += durations1_2
                    operations_accumulator += operations
                    operations1_accumulator += operations1
                    simulazione_accettabile += 1
                    accumula_durata_tutte_colonne += tempo
                    accumula_durata_meno_colonne += tempo1
                    num_confronti_accumulatore += num_confronti
                    conta_sufficienti_accumulatore += conta_sufficienti
                    writer_csv.writerow([4,j, max_column, duration1_1, durations1_2, operations, operations1, i, k])
                    writer_csv.writerow([3, j, max_column, tempo, tempo1, num_confronti_accumulatore, conta_sufficienti_accumulatore, i, k])                    
                    

In [63]:
montecarlo_simulation(num_iterations)

0
Field order: 29
28 28 2
Duplicated rows less columns 1
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2
Duplicated rows less columns 2
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2
Duplicated rows less columns 3
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2
Duplicated rows less columns 4
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2


Duplicated rows less columns 5
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2
Duplicated rows less columns 6
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2
Duplicated rows less columns 7
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2
Duplicated rows less columns 8
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2
Duplicated rows less columns 9
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Non sono uguali
Ordino le colonne
28 28 2
Duplicated rows

KeyboardInterrupt: 